In [ ]:
"""
Created on Mon Jul 24 11:36 2020

This is a script to cut out the T and S in the 50 km in front of the ice front

@author: Clara Burgard
"""

- calculate the distance to the ice front for the small domain in front of the ice shelf
- take the ocean points at distance of ~50 km of the ice front 

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
#from tqdm.notebook import tqdm
from tqdm import tqdm
import gsw
import matplotlib.pyplot as plt
import basal_melt_param.useful_functions as uf
import basal_melt_param.T_S_profile_functions as tspf
import basal_melt_param.melt_functions as meltf
import basal_melt_param.box_functions as bf

from scipy.spatial import cKDTree


import itertools

import distributed
import glob

In [ ]:
client = distributed.Client(n_workers=4, dashboard_address=':8795', local_directory='/tmp', memory_limit='6GB')

In [ ]:
client

In [ ]:
%matplotlib qt5

READ IN THE DATA

In [ ]:
mod = 'CNRM-CM6-1' # 'EPM026','EPM031', 'EPM034'
scenario = 'historical'

if mod == 'CNRM-CM6-1':
    ens_run = 'r1i1p1f2'

In [ ]:
inputpath_data_orig='/bettik/jourdai1/OCEAN_DATA_CMIP6_STEREO/'+mod+'/'
inputpath_profiles='/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/'
inputpath_isf='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_8km/'
inputpath_BedMachine='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'

# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

COMPUTING THE MEAN PROFILES

CONTINENTAL SHELF

In [ ]:
mask_domain_distkm = 50000

If workers don't die (with 12 cores, took approx 1hour), if workers die, divide work by years

In [ ]:

dist_to_front_file = xr.open_mfdataset(inputpath_profiles+'dist_to_ice_front_only_contshelf_8km.nc',chunks={'x': 400, 'y': 400, 'Nisf': 30})
T_ocean_files = xr.open_mfdataset(inputpath_data_orig+'thetao_Oyr_'+mod+'_'+scenario+'_'+ens_run+'_*.nc', chunks={'x': 400, 'y': 400, 'depth': 50, 'time': 5}, parallel=True)
S_ocean_files = xr.open_mfdataset(inputpath_data_orig+'so_Oyr_'+mod+'_'+scenario+'_'+ens_run+'_*.nc', chunks={'x': 400, 'y': 400, 'depth': 50, 'time': 5}, parallel=True)
file_BedMachine_orig = xr.open_mfdataset(inputpath_BedMachine+'BedMachine_v3_aggregated8km_allvars.nc').sel(x=dist_to_front_file.x,y=dist_to_front_file.y).chunk({'x': 400, 'y': 400})

dist_to_front = dist_to_front_file['dist_from_front']
ocean_conc = file_BedMachine_orig['ocean_conc']

In [ ]:
mask_km = dist_to_front <= mask_domain_distkm

In [ ]:
mask_depth = S_ocean_files['so'].isel(time=0).squeeze().drop('time') > 0
mask_all = mask_km & mask_depth

In [ ]:
mask_sum = (mask_all * ocean_conc).sum(['x','y'])
mask_sum = mask_sum.load()
mask_sum.to_dataset(name='mask_sum').to_netcdf(inputpath_profiles + 'mask_sum.nc')

In [ ]:
mask_sum = xr.open_mfdataset(inputpath_profiles + 'mask_sum.nc', chunks={'x': 400, 'y': 400, 'Nisf': 30})

In [ ]:
for tt in T_ocean_files['thetao'].time:
    print(tt.values)
    ds_T_sum = (T_ocean_files['thetao'].sel(time=tt) * mask_km * ocean_conc).sum(['x','y'])
    ds_S_sum = (S_ocean_files['so'].sel(time=tt) * mask_km * ocean_conc).sum(['x','y'])

    ds_T_mean = (ds_T_sum/mask_sum['mask_sum'])
    ds_S_mean = (ds_S_sum/mask_sum['mask_sum'])
    
    ds_T_mean.to_dataset(name='thetao').to_netcdf(inputpath_profiles + mod + '/T_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt.values)[0:4]+'.nc')
    ds_S_mean.to_dataset(name='so').to_netcdf(inputpath_profiles + mod + '/S_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt.values)[0:4]+'.nc')

In [ ]:
#T_S_mean = xr.merge([ds_T_mean,ds_S_mean]).load()

In [ ]:
TS_list = []
for tt in range(yy_start,yy_end+1):
    ds_tt = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_'+str(tt)+'.nc')
    TS_list.append(ds_tt)
TS_ds = xr.concat(TS_list, dim='time')
TS_ds.to_netcdf(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_allyy.nc')